In [11]:
from openai import OpenAI
import base64
import os

In [12]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [13]:
image_path = "shuaidao.png"
base64_image = encode_image(image_path)
data_url = 'data:image/png;base64,'+base64_image

In [ ]:
client = OpenAI(
    base_url='https://api.siliconflow.cn/v1',
    api_key='sk-pawqcnvkrpbjokrapqcqfjnwcpfastyomscrzcscnbbqwywi'
)

response = client.chat.completions.create(
    model="Pro/Qwen/Qwen2.5-VL-7B-Instruct",
    messages=[
        {"role": "user", 
         "content": [
             {'type': 'text',
              'text': '请描述一下这张图片内容，我作为一个社区机器人，我应该做什么'},
             {'type': 'image_url',
              'image_url': {
                  'url': data_url,
                  'detail': 'auto'
              }}
         ]}
    ],
    stream=False  # 启用流式输出
)

In [23]:
print(response.choices[0].message.content)

这张图片中显示了一位女性正向地面躺着的一个人指出方向。这个人看起来似乎在无力挣扎或者受伤中。根据提供的信息，您是一个社区机器人，请注意以下几点：

1. **立即通知求助人**：
   确认求助人是否需要紧急援助，并通过社区平台等工具通知相关求助者或紧急联系方式，如拨打120或911等紧急电话。

2. **确保自身安全**：确认环境安全后，接近有需要帮助的人，避免因救助延误所造成的危险。

3. **提供帮助**：
   - 检查被救助者的意识、言语、呼气、心跳以及脉搏等生命体征。
   - 如果你在现场初次评估有需要帮助的迹象，例如软弱、失去意识或者无法移动等，尝试提供基本的生命急救处置，如保持呼吸道畅通、防止发生呼吸道阻塞，或请求相关人员介入，直至医疗专业人员的到场。
   - 如果确保没有生命危险，尽量派遣医疗专门人员进行咨询处理，或是以负责任的方式将他们送往一个安全的地方，并提醒高铁儿童或需要援助的人到医院。

4. **信息记录和沟通化验**：
   如果此人失去了意识并且无法接触到治疗，应确保其贴身上有相关的医疗信息卡，随后应将患者的已知信息发送给医疗人员，以提高专业救助的准确性和效率。

在紧急情况下，请遵循上述的建议，保持冷静并救助。


In [24]:
import requests
import json

def detect_objects(image_path, api_url="http://localhost:8000/detect/"):
    """
    调用YOLO11检测API
    
    参数:
        image_path: 图像文件路径
        api_url: API地址
        
    返回:
        API返回的检测结果
    """
    # 准备文件
    with open(image_path, "rb") as image_file:
        files = {"file": (image_path, image_file, "image/jpeg")}
        
        # 发送POST请求到API
        response = requests.post(api_url, files=files)
        
    # 检查响应
    if response.status_code == 200:
        # 解析JSON响应
        results = response.json()
        print(f"检测到 {results['total']} 个对象:")
        
        for det in results["detections"]:
            print(f"- 类别: {det['class_name']}, 置信度: {det['confidence']:.2f}")
            
        return results
    else:
        print(f"错误: {response.status_code}, {response.text}")
        return None

In [25]:
if __name__ == "__main__":
    # 替换为你的图像路径
    image_path = "shuaidao.jpg"
    
    # 替换为你的API地址（如果在远程服务器上）
    # api_url = "http://your_server_ip:8000/detect/"
    api_url = "http://localhost:8000/detect/"
    
    results = detect_objects(image_path, api_url)

检测到 3 个对象:
- 类别: person, 置信度: 0.93
- 类别: person, 置信度: 0.92
- 类别: couch, 置信度: 0.82
